# T_HUMR-1 Generation Analysis
Analysis of generation patterns, stops, and weekly statistics

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set style for better visualization
plt.style.use('seaborn')
sns.set_palette("husl")

In [ ]:
# Read the data
df = pd.read_csv('T_HUMR-1_bidding_strategy.csv')
df['year_month'] = pd.to_datetime(df['year_month'])

# Focus on last 24 months
last_date = df['year_month'].max()
start_date = last_date - pd.DateOffset(months=24)
recent_df = df[df['year_month'] >= start_date]

## Generation Volumes Over Time

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(recent_df['year_month'], recent_df['total_offer_volume_mw'], marker='o')
plt.axhline(y=0, color='r', linestyle='--', alpha=0.3)
plt.title('T_HUMBR-1 Generation Volume (Last 24 Months)')
plt.xlabel('Date')
plt.ylabel('Total Offer Volume (MW)')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()

# Highlight generation stops
stops = recent_df[recent_df['total_offer_volume_mw'] < 0]
plt.scatter(stops['year_month'], stops['total_offer_volume_mw'], 
           color='red', s=100, label='Generation Stops')
plt.legend()
plt.show()

## Weekly Generation Statistics

In [ ]:
# Convert monthly data to weekly statistics
recent_df.set_index('year_month', inplace=True)
weekly_df = recent_df.resample('W')['total_offer_volume_mw'].agg(['min', 'median', 'max']).reset_index()

# Plot weekly statistics
plt.figure(figsize=(15, 8))
plt.fill_between(weekly_df['year_month'], weekly_df['min'], weekly_df['max'], 
                 alpha=0.3, label='Min-Max Range')
plt.plot(weekly_df['year_month'], weekly_df['median'], 'b-', 
         label='Median Generation', linewidth=2)

plt.title('T_HUMBR-1 Weekly Generation Statistics')
plt.xlabel('Date')
plt.ylabel('Generation Volume (MW)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Generation Stops Analysis

In [ ]:
# Analyze generation stops
stops = recent_df[recent_df['total_offer_volume_mw'] < 0].copy()

# Create a plot showing stops and corresponding market prices
fig, ax1 = plt.subplots(figsize=(15, 8))

# Plot offer prices
ax1.set_xlabel('Date')
ax1.set_ylabel('Offer Price (GBP/MWh)', color='tab:blue')
ax1.plot(recent_df.index, recent_df['avg_offer_price_gbp_mwh'], 
         color='tab:blue', label='Offer Price')
ax1.tick_params(axis='y', labelcolor='tab:blue')

# Create second y-axis for generation volume
ax2 = ax1.twinx()
ax2.set_ylabel('Stop Volume (MW)', color='tab:red')
ax2.scatter(stops.index, stops['total_offer_volume_mw'], 
            color='tab:red', s=100, label='Generation Stops')
ax2.tick_params(axis='y', labelcolor='tab:red')

plt.title('Generation Stops vs Market Prices')
fig.tight_layout()
plt.show()

## Price Spread During Generation Stops

In [ ]:
# Calculate price spread
recent_df['price_spread'] = recent_df['avg_offer_price_gbp_mwh'].abs() + recent_df['avg_bid_price_gbp_mwh'].abs()

plt.figure(figsize=(15, 8))
plt.bar(recent_df.index, recent_df['price_spread'], alpha=0.3, label='Price Spread')

# Highlight spreads during stops
stops_spread = recent_df[recent_df['total_offer_volume_mw'] < 0]['price_spread']
plt.bar(stops_spread.index, stops_spread, color='red', label='Spread During Stops')

plt.title('Price Spread Analysis (Offer-Bid)')
plt.xlabel('Date')
plt.ylabel('Price Spread (GBP/MWh)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()